# BMKG - Assignment 3

This notebook intends to automatically generate a SCHEMA for any given KG.

In [1]:
from operator import itemgetter
import getpass
import os

from typing import Any

from rdflib import Graph

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import get_buffer_string
from langchain_openai import OpenAI

from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS

from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Provide your OpenAI API Key")

Provide your OpenAI API Key········


1) Get the RDF graph

In [3]:
WD = Namespace("http://www.wikidata.org/entity/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
ex = Namespace("https://www.example.org/")
wdt = Namespace("https://www.wikidata.org/prop/direct/")

In [4]:
sparql_query = """
SELECT DISTINCT ?drug ?drugLabel ?disease ?diseaseLabel ?cause ?causeLabel ?symptom ?symptomLabel ?sideEffect ?sideEffectLabel ?gene ?geneLabel
WHERE {
  ?drug wdt:P31 wd:Q12140 .  # medicine
  ?drug wdt:P2175 ?disease .    # treats disease
  OPTIONAL { ?disease wdt:P828 ?cause } .  # cause of disease
  OPTIONAL { ?disease wdt:P780 ?symptom } .  # symptom/complaint of disease
  OPTIONAL { ?drug wdt:P780 ?sideEffect } .  # potential side effect
  OPTIONAL { ?disease wdt:P1057 ?gene } .  # gene associated with disease
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
"""

In [39]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Create an RDF graph
g = Graph()

# Bind namespaces
g.bind("wd", WD)
g.bind("wdt", wdt)
g.bind("rdfs", RDFS)

# Process query results
for result in results["results"]["bindings"]:
    drug = URIRef(result["drug"]["value"])
    disease = URIRef(result["disease"]["value"])

    # Add RDF types
    g.add((drug, RDF.type, ex.Drug))
    g.add((disease, RDF.type, ex.Disease))
    if "cause" in result:
        cause = URIRef(result["cause"]["value"])
        g.add((cause, RDF.type, ex.Cause))
    if "symptom" in result:
        symptom = URIRef(result["symptom"]["value"])
        g.add((symptom, RDF.type, ex.Symptom))
    if "sideEffect" in result:
        side_effect = URIRef(result["sideEffect"]["value"])
        g.add((side_effect, RDF.type, ex.SideEffect))
    if "gene" in result:
        gene = URIRef(result["gene"]["value"])
        g.add((gene, RDF.type, ex.Gene))

    # Add labels
    g.add((drug, RDFS.label, Literal(result["drugLabel"]["value"])))
    g.add((disease, RDFS.label, Literal(result["diseaseLabel"]["value"])))
    if "cause" in result:
        g.add((cause, RDFS.label, Literal(result["causeLabel"]["value"])))
    if "symptom" in result:
        g.add((symptom, RDFS.label, Literal(result["symptomLabel"]["value"])))
    if "sideEffect" in result:
        g.add((side_effect, RDFS.label, Literal(result["sideEffectLabel"]["value"])))
    if "gene" in result:
        g.add((gene, RDFS.label, Literal(result["geneLabel"]["value"])))

    # Add relationships
    g.add((drug, ex.treats, disease))
    if "cause" in result:
        g.add((disease, ex.hasCause, cause))
    if "symptom" in result:
        g.add((disease, ex.hasSymptom, symptom))
    if "sideEffect" in result:
        g.add((drug, ex.hasSideEffect, side_effect))
    if "gene" in result:
        g.add((disease, ex.associatedGene, gene))

# Serialize the graph to a TTL file
g.serialize(destination='medical_graph.ttl', format='turtle')


<Graph identifier=N448cbf7e146b4221b38c4c40704eee2b (<class 'rdflib.graph.Graph'>)>

In [4]:
#opening the files to use with the llm
with open('wikidata_drug_disease_schema.ttl', 'r') as file:
    schema = file.read()
    
graph = Graph()
graph = graph.parse("medical_graph.ttl")

print("Number of triples: ", len(graph))

Number of triples:  1873


In [5]:
llm = OpenAI(temperature=0)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# Create the memory object that is used to add messages
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)
# Add a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Prompt to reformulate the question using the chat history
reform_template = """Given the following chat history and a follow up question,
rephrase the follow up question to be a standalone straightforward question, in its original language.
Do not answer the question! Just rephrase reusing information from the chat history.

Chat History:
{chat_history}
Follow up input:
{question}

Standalone question:
"""
REFORM_QUESTION_PROMPT = PromptTemplate.from_template(reform_template)

breakdown_template = """ Given the following question, break it down to identify the known variables. Format these variables as follows:
Name --> Variable Type

Lastly, identify what the question is looking for (including the name, if provided). Only use the information presented in the question, nothing else.

Question:
{question}

Breakdown:
"""
BREAKDOWN_PROMPT = PromptTemplate.from_template(breakdown_template)

# Prompt to ask to answer the reformulated question
answer_template = """Construct a valid SPARQL query based on the provided breakdown, ensuring that labels are compared in lowercase using LCASE and not directly added as triples.

Based on the breakdown, the query should only retrieve the information we are looking for. Use the provided schema, although the query should not include everything!

Schema:
{schema}

Breakdown:
{breakdown}

Query:
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

In [11]:
# Reformulate the question using chat history
reformulated_question = {
    "reformulated_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | REFORM_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Breakdown the question to identify what we are looking for and what we know
question_breakdown = {
    "breakdown" : {
        "question": lambda x: print("💭 Reformulated question:", x["reformulated_question"]) or x["reformulated_question"],
    }
    | BREAKDOWN_PROMPT
    | llm
    | StrOutputParser(),
}
final_inputs = {
    "schema": lambda x: schema,
    "breakdown": lambda x: print("💭 Question breakdown:\n", x["breakdown"]) or x["breakdown"],
}
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
}
# Put the chain together
final_chain = loaded_memory | reformulated_question | question_breakdown | answer

def stream_chain(final_chain, memory: ConversationBufferMemory, inputs_list: list[dict[str, str]]) -> dict[str, Any]:
    """Ask questions, stream the answer output, and return the answers."""
    output = {"answer": []}
    for inputs in inputs_list:
        answer_output = ""
        for chunk in final_chain.stream(inputs):
            if "answer" in chunk:
                answer_output += chunk["answer"]
                print(chunk["answer"], end="", flush=True)
        
        output["answer"].append(answer_output)
        # Add messages to chat history
        memory.save_context(inputs, {"answer": answer_output})
    
    return output

In [41]:
questions = [
    {"question": "What disease is caused by 'food?"},
]
output = stream_chain(final_chain, memory, questions)

💭 Reformulated question: What disease is caused by 'food'?
💭 Question breakdown:
 
Known Variables:
- Disease --> String
- Food --> String

Question is looking for:
- The name of the disease caused by 'food'
PREFIX ns1: <https://www.example.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?diseaseLabel
WHERE {
    ?diseaseURI a ns1:Disease ;
        rdfs:label ?diseaseLabel ;
        ns1:hasCause ?causeURI .
    ?causeURI a ns1:Cause ;
        rdfs:label ?causeLabel .
    FILTER(LCASE(?causeLabel) = LCASE("food"))
}

In [42]:
import pandas as pd
import rdflib

from IPython.display import display, HTML
from pygments import highlight
from pygments.lexers import SparqlLexer
from pygments.formatters import HtmlFormatter

def run_query(graph, query, entity=None):
    # Execute the SPARQL query
    results = graph.query(query, initBindings={'entity': entity})
    
    # Display the SPARQL query
    formatted_query = highlight(query, SparqlLexer(), HtmlFormatter(style='solarized-dark', full=True, nobackground=True))
    display(HTML(formatted_query))
    
    # Convert results to a Pandas DataFrame
    res_list = []
    for row in results:
        res_list.append([str(item) for item in row])
    df = pd.DataFrame(res_list, columns=[str(var) for var in results.vars]) if len(res_list) > 0 else pd.DataFrame()

    # Display the DataFrame as a table in Jupyter Notebook
    display(HTML(df.to_html()))
    
    if(len(df) > 1):
        return df.iloc[0, 0]


In [43]:
for answer_output in output["answer"]:
    run_query(graph, answer_output)

PREFIX ns1: <https://www.example.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?diseaseLabel
WHERE {
    ?diseaseURI a ns1:Disease ;
        rdfs:label ?diseaseLabel ;
        ns1:hasCause ?causeURI .
    ?causeURI a ns1:Cause ;
        rdfs:label ?causeLabel .
    FILTER(LCASE(?causeLabel) = LCASE("food"))
}


,diseaseLabel
0,food allergy


None


In [19]:
def get_uri_from_label(label, graph):
    """
    Given a human-readable label and a graph, return the URI corresponding to the label.
    If multiple URIs exist for a given label, this function returns the first match.
    """
    sparql_query = """
    SELECT ?entity WHERE {
        ?entity rdfs:label ?label .
        FILTER (LCASE(STR(?label)) = LCASE(?input_label))
    }
    LIMIT 1
    """
    query_result = graph.query(sparql_query, initBindings={'input_label': rdflib.Literal(label)})
    for row in query_result:
        return row[0]  # Return the first (and hopefully only) URI that matches the label
    return None  # Return None if no matching URI is foun

In [50]:
#To combine our multi-hop reasoning with our llm, we ask the llm two questions, and we try to find the link between them.
import rdflib
import networkx as nx
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph

def multi_hop_questioning(question1, question2, graph, final_chain, memory):

    #now we have our questions ready, we find our answers
    question1 = [
            {"question" : question1},
    ]
    output1 = stream_chain(final_chain, memory, question1)
    question2 = [
            {"question": question2},
    ]
    output2 = stream_chain(final_chain, memory, question2)
    
    for answer_output in output1["answer"]:
        print(answer_output)
        answer1 = run_query(graph, answer_output)
        
    for answer_output in output2["answer"]:
        print(answer_output)
        answer2 = run_query(graph, answer_output)

    dg = rdflib_to_networkx_digraph(graph)
    
    query_entity = get_uri_from_label(answer1, graph)
    target_relation = get_uri_from_label(answer2, graph)
    
    visited = set()
    queue = [(query_entity, [])]  # Start with the query entity and an empty relation path
    while queue:
        entity, relation_path = queue.pop(0)
        if entity == target_relation:
            return relation_path
        if entity not in visited:
            visited.add(entity)
            for neighbor in dg.neighbors(entity):
                new_path = relation_path + [(entity, neighbor)]
                queue.append((neighbor, new_path))
    return None

In [51]:
q1 = "What causes does the disease 'food allergy' have'?"
q2 = "What disease is caused by 'food'?"

x = multi_hop_questioning(q1, q2, graph, final_chain, memory)

💭 Reformulated question: What causes does the disease 'food allergy' have?
💭 Question breakdown:
 
Known Variables:
- Disease: String

Question is looking for:
- Causes of food allergy
PREFIX ns1: <https://www.example.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?causeLabel
WHERE {
    ?diseaseURI a ns1:Disease ;
        rdfs:label ?diseaseLabel ;
        ns1:hasCause ?causeURI .
    ?causeURI a ns1:Cause ;
        rdfs:label ?causeLabel .
    FILTER(LCASE(?diseaseLabel) = "food allergy")
}💭 Reformulated question: What disease is caused by 'food'?
💭 Question breakdown:
 
Known Variables:
- Disease --> String
- Food --> String

Question is looking for:
- The name of the disease caused by 'food'
PREFIX ns1: <https://www.example.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?diseaseLabel
WHERE {
    ?diseaseURI a ns1:Disease ;
        rdfs:label ?diseaseLabe

,causeLabel
0,food
1,food allergen


PREFIX ns1: <https://www.example.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>

SELECT ?diseaseLabel
WHERE {
    ?diseaseURI a ns1:Disease ;
        rdfs:label ?diseaseLabel ;
        ns1:hasCause ?causeURI .
    ?causeURI a ns1:Cause ;
        rdfs:label ?causeLabel .
    FILTER(LCASE(?causeLabel) = LCASE("food"))
}


,diseaseLabel
0,food allergy


In [49]:
print(x)

None
